<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City</font></h1>

## Introduction

In this assignment, addresses are converted into their equivalent latitude and longitude values. Also, the Foursquare API is used to explore neighborhoods in Toronto City. The **explore** function is used to get the most common venue categories in each neighborhood, and then this feature is used to group the neighborhoods into clusters. The *k*-means clustering algorithm is used to complete this task. Finally, the Folium library is used to visualize the neighborhoods in New York City and their emerging clusters.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="item1">Extract Data, Create Dataset and Explore Dataset</a>

2. <a href="#item2">Find latitude, longitude by postal codes</a>

3. <a href="#item3">Explore Neighborhoods in Toronto City</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## <a id='item1'></a>1. Extract Data, Create Dataset and Explore Dataset

 Import the library we will be using to connect to the Wikipedia page and fetch the contents of that page:

In [2]:
# import the library we use to open URLs
import urllib.request

Specify the URL of the Wikipedia page we are looking to scrape:

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

Using the urllib.request library, we want to query the page and put the HTML data into a variable:

In [4]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

Import the functions from Beautiful Soup which will let us parse and work with the HTML we fetched from our Wiki page:

In [5]:
#!conda install -c conda-forge bs4 --yes
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [6]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page)

We know the data resides within an HTML table so firstly we send Beautiful Soup off to retrieve all instances of the \<table\> tag within the page and add them to an array called all_tables:

In [7]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
#all_tables

Looking through the output of ”all_tables” we can again see that the class id of our chosen table is ”wikitable sortable”. We can use this to get BS to only bring back the table data for this particular table and keep that in a variable called ”right_table“:

In [8]:
right_table=soup.find('table', class_='wikitable sortable')
#right_table

Parse the data and make lists of Postal Code, Borough, Neighborhood

In [9]:
Code=[]
Borough=[]
Neighborhood=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        if(str(cells[1].find(text=True).replace('\n',''))==str("Not assigned")):
            continue;
        Code.append(cells[0].find(text=True).replace('\n',''))
        Borough.append(cells[1].find(text=True).replace('\n',''))
        if(str(cells[2].find(text=True).replace('\n',''))==str("Not assigned")):
            Neighborhood.append(cells[1].find(text=True).replace('\n',''))
        else:
            Neighborhood.append(cells[2].find(text=True).replace('\n',''))

Make Pandas dataframe and print shape, dataframe contents

In [10]:
df=pd.DataFrame(Code,columns=['Postal Code'])
df['Borough']=Borough
df['Neighborhood']=Neighborhood
print("No of rows in dataframe : " +str(df.shape[0])+", df.shape = "+str(df.shape))
df.head()

No of rows in dataframe : 103, df.shape = (103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 2. Find latitude, longitude by postal codes

Read csv file containing coordinates data

In [11]:
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframes by Postal Code

In [12]:
neighborhoods = pd.merge(df,geo,on='Postal Code')
neighborhoods = neighborhoods[neighborhoods.Borough.str.contains("Toronto")]
print(neighborhoods.shape)
neighborhoods.reset_index(inplace=True)
neighborhoods.head()

(39, 5)


,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


## 3. Explore Neighborhoods in Toronto City

In [13]:
#!conda install -c conda-forge geocoder --yes
#import geocoder # import geocoder

In [14]:
# initialize your variable to None
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
#  g = geocoder.google('Toronto, Ontario')
#  lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]#43.6532
#longitude = lat_lng_coords[1]#-79.3832

In [15]:
latitude = 43.6532
longitude = -79.3832

In [16]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Define Foursquare Credentials and Version

In [17]:
CLIENT_ID = 'KVKM0AUUBGIAWTJXVJERP00QDVYM34TW22VYU1GPIEBKLECQ' # your Foursquare ID
CLIENT_SECRET = 'AY00TWMDXDNSW50N4DEWHZWBSEANF3NB0TKOHD21A14Z3S1O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KVKM0AUUBGIAWTJXVJERP00QDVYM34TW22VYU1GPIEBKLECQ
CLIENT_SECRET:AY00TWMDXDNSW50N4DEWHZWBSEANF3NB0TKOHD21A14Z3S1O


Test if Foursquare API is working

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KVKM0AUUBGIAWTJXVJERP00QDVYM34TW22VYU1GPIEBKLECQ&client_secret=AY00TWMDXDNSW50N4DEWHZWBSEANF3NB0TKOHD21A14Z3S1O&v=20180605&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [20]:
results = requests.get(url).json()
#results

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called neighborhood_venues.

In [23]:
neighborhood_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Let's check the size of the resulting dataframe

In [24]:
print(neighborhood_venues.shape)
neighborhood_venues.head()

(1620, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub


Let's check how many venues were returned for each neighborhood

In [25]:
neighborhood_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,64,64,64,64,64,64


Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 237 uniques categories.


### Analyze Each Neighborhood

In [27]:
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1

And let's examine the new dataframe size.

In [28]:
neighborhood_onehot.shape

(1620, 237)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [29]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0,0.0,0.0,0.017857,0.000000,0.0,0.0,0.0,0.0000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.000000,0.017857,0.0,0.0000,0.0,0.0000,0.0,0.017857,0.0000,0.00000,0.0,0.00000,0.000000,0.0,0.0000,0.0,0.035714,0.0,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.000000,0.017857,0.053571,0.089286,0.0,0.0,0.0,0.0,0.0,0.017857,0.000000,0.017857,0.000000,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.017857,0.000000,0.0,0.017857,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.000000,0.017857,0.0000,0.0,0.0,0.017857,0.0,0.0,0.0

Let's confirm the new size

In [31]:
neighborhood_grouped.shape

(39, 237)

Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in neighborhood_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighborhood_grouped[neighborhood_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    break

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4            Beer Bar  0.04




Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Breakfast Spot,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Bakery,Convenience Store,Nightclub,Italian Restaurant,Restaurant,Intersection,Burrito Place
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Comic Shop,Fast Food Restaurant,Farmers Market,Burrito Place,Auto Workshop,Spa,Restaurant,Pizza Place,Butcher
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Department Store,Salad Place,Thai Restaurant,Burger Joint,Bubble Tea Shop


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [49]:
# set number of clusters
kclusters = 3

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2,
       2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_merged = neighborhoods_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

neighborhoods_merged.dropna(axis = 0, how ='any',inplace=True)
neighborhoods_merged[0:5]# check the last columns!

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Beer Store
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,2,Coffee Shop,Sushi Restaurant,Diner,Sandwich Place,Bar,Smoothie Shop,Beer Bar,Music Venue,Italian Restaurant,Sculpture Garden
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Diner,Pizza Place
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Café,Coffee Shop,American Restaurant,Restaurant,Cocktail Bar,Gastropub,Hotel,Department Store,Clothing Store,Italian Restaurant
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Asian Restaurant,Pub,Trail,Health Food Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_merged['Latitude'], neighborhoods_merged['Longitude'], neighborhoods_merged['Neighborhood'], neighborhoods_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, assign a name to each cluster.

#### Cluster 1

In [38]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 0, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5V,-79.39442,0,Airport Service,Airport Terminal,Airport Lounge,Plane,Boat or Ferry,Rental Car Location,Sculpture Garden,Harbor / Marina,Boutique,Airport Gate


#### Cluster 2

In [39]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 1, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,-79.360636,1,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Café,Theater,Cosmetics Shop,Beer Store
1,M7A,-79.389494,1,Coffee Shop,Sushi Restaurant,Diner,Sandwich Place,Bar,Smoothie Shop,Beer Bar,Music Venue,Italian Restaurant,Sculpture Garden
2,M5B,-79.378937,1,Clothing Store,Coffee Shop,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Italian Restaurant,Café,Middle Eastern Restaurant,Diner,Pizza Place
3,M5C,-79.375418,1,Café,Coffee Shop,American Restaurant,Restaurant,Cocktail Bar,Gastropub,Hotel,Department Store,Clothing Store,Italian Restaurant
5,M5E,-79.373306,1,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Beer Bar,Seafood Restaurant,Restaurant,Bakery,Breakfast Spot,Irish Pub
6,M5G,-79.387383,1,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Department Store,Salad Place,Thai Restaurant,Burger Joint,Bubble Tea Shop
8,M5H,-79.384568,1,Coffee Shop,Café,Restaurant,Gym,Deli / Bodega,Hotel,Thai Restaurant,Salad Place,Cosmetics Shop,Steakhouse
10,M5J,-79.381752,1,Coffee Shop,Aquarium,Hotel,Café,Scenic Lookout,Sporting Goods Shop,Brewery,Restaurant,Fried Chicken Joint,Italian Restaurant
11,M6J,-79.419750,1,Bar,Restaurant,Asian Restaurant,Vietnamese Restaurant,Men's Store,Vegetarian / Vegan Restaurant,Coffee Shop,Café,Korean Restaurant,Cupcake Shop
12,M4K,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Cosmetics Shop,Indian Restaurant,Japanese Restaurant


#### Cluster 3

In [40]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 2, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,M5N,-79.416936,2,Garden,Women's Store,Cupcake Shop,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


#### Cluster 4

In [41]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 3, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,M4W,-79.377529,3,Park,Trail,Playground,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


#### Cluster 5

In [42]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 4, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,M4N,-79.38879,4,Park,Swim School,Lawyer,Bus Line,College Rec Center,Colombian Restaurant,College Auditorium,Doner Restaurant,Dog Run,Distribution Center


#### Cluster 6

In [43]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 5, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,M5P,-79.411307,5,Trail,Sushi Restaurant,Jewelry Store,Park,Bus Line,Dessert Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store


#### Cluster 7

In [44]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 6, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,M6G,-79.422564,6,Grocery Store,Café,Park,Athletics & Sports,Diner,Restaurant,Candy Store,Italian Restaurant,Nightclub,Coffee Shop
9,M6H,-79.442259,6,Pharmacy,Bakery,Pet Store,Bar,Supermarket,Café,Recording Studio,Middle Eastern Restaurant,Bank,Park
14,M6K,-79.428191,6,Café,Breakfast Spot,Coffee Shop,Bakery,Convenience Store,Nightclub,Italian Restaurant,Restaurant,Intersection,Burrito Place
17,M4M,-79.340923,6,Café,Coffee Shop,Gastropub,American Restaurant,Brewery,Bakery,Italian Restaurant,Park,Seafood Restaurant,Sandwich Place
24,M5R,-79.405678,6,Sandwich Place,Café,Coffee Shop,Pharmacy,Pub,BBQ Joint,Middle Eastern Restaurant,History Museum,Indian Restaurant,Burger Joint
27,M5S,-79.400049,6,Café,Restaurant,Bookstore,Japanese Restaurant,Italian Restaurant,Bar,Bakery,Sushi Restaurant,Sandwich Place,College Gym


#### Cluster 8

In [45]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 7, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,M4P,-79.390197,7,Sandwich Place,Breakfast Spot,Hotel,Department Store,Convenience Store,Pizza Place,Food & Drink Shop,Gym,Park,Diner


#### Cluster 9

In [46]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 8, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4T,-79.38316,8,Park,Restaurant,Playground,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


#### Cluster 10

In [47]:
neighborhoods_merged.loc[neighborhoods_merged['Cluster Labels'] == 9, neighborhoods_merged.columns[[1] + list(range(5, neighborhoods_merged.shape[1]))]]

,Postal Code,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M4E,-79.293031,9,Asian Restaurant,Pub,Trail,Health Food Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
